# 🔍 Aprendizaje Automático No Supervisado
## Descubrimiento de Patrones Latentes en Datos

<a href="https://colab.research.google.com/github/yourusername/ml-course/blob/main/01_Sistemas_aprendizaje_automatico/02_Ml_no_supervisado/ml_no_supervisado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---

## 📋 Resumen del Notebook

Este notebook explora el **aprendizaje no supervisado**, un paradigma fundamental del ML que busca descubrir estructuras inherentes en datos sin etiquetas predefinidas.

### 🎯 Objetivos de Aprendizaje

1. **Algoritmos de Clustering**:
   - K-Means: Particionamiento iterativo del espacio de datos
   - Clustering Jerárquico: Construcción de dendrogramas
   - DBSCAN: Clustering basado en densidad
   - Gaussian Mixture Models (GMM): Enfoque probabilístico

2. **Reducción de Dimensionalidad**:
   - PCA (Principal Component Analysis): Análisis de componentes principales
   - t-SNE: Visualización no lineal de datos de alta dimensión

3. **Detección de Anomalías**:
   - Isolation Forest: Detección de outliers
   - Local Outlier Factor (LOF)

4. **Aplicaciones Prácticas**:
   - Segmentación de clientes
   - Compresión de imágenes
   - Visualización de datos complejos
   - Detección de fraude

### 📊 Contenido

- Fundamentos teóricos con formulaciones matemáticas
- Implementaciones prácticas con Scikit-learn
- Visualizaciones interactivas
- Comparación de algoritmos
- Casos de uso reales

---

In [ ]:
# Configuración del entorno
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_blobs, make_moons, make_circles, load_digits, load_wine
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import cdist
import warnings

warnings.filterwarnings('ignore')
sns.set_style('darkgrid')
plt.rcParams['figure.dpi'] = 100

print("✅ Entorno configurado correctamente")
print(f"NumPy: {np.__version__} | Pandas: {pd.__version__}")
print(f"Scikit-learn disponible con algoritmos de clustering y reducción de dimensionalidad")

## 1. Fundamentos del Aprendizaje No Supervisado

### 🎓 Marco Conceptual

El aprendizaje no supervisado trabaja con conjuntos de datos $\mathcal{D} = \{x_i\}_{i=1}^{n}$ donde $x_i \in \mathbb{R}^d$ son observaciones **sin etiquetas** asociadas.

#### Objetivos Principales:

1. **Descubrimiento de Estructura**: Identificar agrupamientos naturales o patrones de similitud
2. **Reducción de Dimensionalidad**: Encontrar representaciones compactas preservando información relevante
3. **Detección de Anomalías**: Identificar observaciones que se desvían de patrones normales
4. **Aprendizaje de Representaciones**: Descubrir características latentes

#### Desafíos Metodológicos:

- **Ausencia de Ground Truth**: Sin etiquetas verdaderas, la evaluación es fundamentalmente subjetiva
- **Definición de Similitud**: La noción de similitud puede ser específica del dominio
- **Escalabilidad**: Muchos algoritmos tienen complejidad computacional prohibitiva
- **Interpretabilidad**: Los resultados requieren validación mediante conocimiento del dominio

---

## 2. Clustering: K-Means

### 📐 Formulación Matemática

K-Means minimiza la **inercia intra-cluster** (within-cluster sum of squares):

$$J = \sum_{k=1}^{K}\sum_{x_i \in C_k}\|x_i - \mu_k\|^2$$

donde $\mu_k$ es el centroide del cluster $C_k$.

### Algoritmo de Lloyd:

1. **Inicialización**: Seleccionar $K$ centroides (K-Means++ recomendado)
2. **Asignación**: Asignar cada punto al centroide más cercano
3. **Actualización**: Recalcular centroides como media de puntos asignados
4. **Iteración**: Repetir hasta convergencia

### ✅ Ventajas:
- Eficiencia: $O(nKdi)$ (n=muestras, K=clusters, d=dimensiones, i=iteraciones)
- Escalabilidad a grandes datasets
- Convergencia garantizada a mínimo local

### ⚠️ Limitaciones:
- Requiere especificar $K$ a priori
- Asume clusters esféricos de tamaño similar
- Sensible a outliers y inicialización

In [ ]:
# Ejemplo Práctico: K-Means en Datasets Sintéticos

print("🔬 Generando datasets sintéticos con diferentes estructuras...\n")

# Crear tres tipos de datasets
np.random.seed(42)

# Dataset 1: Clusters esféricos (ideal para K-Means)
X_blobs, y_blobs_true = make_blobs(n_samples=300, centers=4, cluster_std=0.6, random_state=42)

# Dataset 2: Clusters en forma de lunas (desafiante para K-Means)
X_moons, y_moons_true = make_moons(n_samples=300, noise=0.05, random_state=42)

# Dataset 3: Clusters circulares concéntricos
X_circles, y_circles_true = make_circles(n_samples=300, noise=0.05, factor=0.5, random_state=42)

datasets = [
    (X_blobs, y_blobs_true, "Clusters Esféricos"),
    (X_moons, y_moons_true, "Clusters en Lunas"),
    (X_circles, y_circles_true, "Clusters Circulares")
]

# Aplicar K-Means a cada dataset
fig, axes = plt.subplots(3, 3, figsize=(16, 14))

for idx, (X, y_true, name) in enumerate(datasets):
    # Datos originales
    axes[idx, 0].scatter(X[:, 0], X[:, 1], c=y_true, cmap='viridis', s=50, alpha=0.7, edgecolors='k')
    axes[idx, 0].set_title(f'{name}\n(Ground Truth)', fontweight='bold', fontsize=11)
    axes[idx, 0].set_xlabel('Feature 1')
    axes[idx, 0].set_ylabel('Feature 2')
    
    # K-Means clustering
    n_clusters = len(np.unique(y_true))
    kmeans = KMeans(n_clusters=n_clusters, init='k-means++', random_state=42, n_init=10)
    y_kmeans = kmeans.fit_predict(X)
    
    axes[idx, 1].scatter(X[:, 0], X[:, 1], c=y_kmeans, cmap='plasma', s=50, alpha=0.7, edgecolors='k')
    axes[idx, 1].scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], 
                        c='red', marker='X', s=300, edgecolors='black', linewidths=2, 
                        label='Centroides', zorder=10)
    axes[idx, 1].set_title(f'K-Means (K={n_clusters})\nInercia: {kmeans.inertia_:.2f}', 
                          fontweight='bold', fontsize=11)
    axes[idx, 1].set_xlabel('Feature 1')
    axes[idx, 1].legend()
    
    # Elbow method para determinar K óptimo
    inertias = []
    silhouette_scores = []
    K_range = range(2, 11)
    
    for k in K_range:
        km = KMeans(n_clusters=k, init='k-means++', random_state=42, n_init=10)
        km.fit(X)
        inertias.append(km.inertia_)
        silhouette_scores.append(silhouette_score(X, km.labels_))
    
    ax_twin = axes[idx, 2].twinx()
    
    axes[idx, 2].plot(K_range, inertias, 'bo-', linewidth=2, markersize=8, label='Inercia')
    axes[idx, 2].axvline(x=n_clusters, color='red', linestyle='--', linewidth=2, alpha=0.7, label=f'K óptimo={n_clusters}')
    axes[idx, 2].set_xlabel('Número de Clusters (K)')
    axes[idx, 2].set_ylabel('Inercia', color='b')
    axes[idx, 2].tick_params(axis='y', labelcolor='b')
    axes[idx, 2].grid(True, alpha=0.3)
    
    ax_twin.plot(K_range, silhouette_scores, 'gs-', linewidth=2, markersize=8, label='Silhouette')
    ax_twin.set_ylabel('Silhouette Score', color='g')
    ax_twin.tick_params(axis='y', labelcolor='g')
    
    axes[idx, 2].set_title(f'Elbow Method - {name}', fontweight='bold', fontsize=11)
    axes[idx, 2].legend(loc='upper right')
    ax_twin.legend(loc='center right')

plt.tight_layout()
plt.show()

print("\n📊 Análisis de Resultados:")
print("=" * 70)
print("✅ Clusters Esféricos: K-Means funciona perfectamente")
print("⚠️  Clusters en Lunas: K-Means falla (geometría no convexa)")
print("⚠️  Clusters Circulares: K-Means no captura estructura concéntrica")
print("\n💡 Lección: K-Means es efectivo solo para clusters esféricos y bien separados")

## 3. Clustering Jerárquico

### 🌳 Construcción de Jerarquías

El clustering jerárquico construye una **jerarquía de agrupamientos** representada mediante un **dendrograma**, permitiendo visualización a múltiples escalas.

#### Enfoque Agglomerative (Bottom-Up):
1. Inicializar cada punto como cluster individual
2. Iterativamente fusionar los dos clusters más similares
3. Continuar hasta obtener un único cluster

#### Criterios de Enlace (Linkage):

- **Single Linkage**: $d(C_i, C_j) = \min_{x \in C_i, y \in C_j} d(x,y)$ 
  - Puede producir efecto "encadenamiento"
  
- **Complete Linkage**: $d(C_i, C_j) = \max_{x \in C_i, y \in C_j} d(x,y)$
  - Produce clusters más compactos
  
- **Average Linkage**: $d(C_i, C_j) = \frac{1}{|C_i||C_j|}\sum_{x \in C_i}\sum_{y \in C_j} d(x,y)$
  - Balance entre single y complete
  
- **Ward's Method**: Minimiza incremento en varianza intra-cluster
  - Más robusto, recomendado en general

### ✅ Ventajas:
- No requiere especificar $K$ a priori
- Dendrograma visualiza estructura jerárquica completa
- Determinístico (sin inicialización aleatoria)

### ⚠️ Limitaciones:
- Complejidad: $O(n^2\log n)$ 
- No escala a datasets masivos
- Decisiones de fusión son irrevocables

In [ ]:
# Ejemplo Práctico: Clustering Jerárquico con Dendrogramas

print("🌳 Aplicando Clustering Jerárquico con diferentes métodos de enlace...\n")

# Generar datos con clusters claros
np.random.seed(42)
X_hier, y_hier = make_blobs(n_samples=150, centers=3, cluster_std=0.8, random_state=42)

# Métodos de enlace a comparar
linkage_methods = ['ward', 'complete', 'average', 'single']

fig = plt.figure(figsize=(18, 12))

# Crear dendrogramas para cada método
for idx, method in enumerate(linkage_methods):
    # Subplot para dendrograma
    ax1 = plt.subplot(2, 4, idx + 1)
    
    # Calcular linkage
    Z = linkage(X_hier, method=method)
    
    # Crear dendrograma
    dendrogram(Z, ax=ax1, color_threshold=0.3*max(Z[:, 2]))
    ax1.set_title(f'Dendrograma\n{method.capitalize()} Linkage', fontweight='bold', fontsize=11)
    ax1.set_xlabel('Índice de Muestra')
    ax1.set_ylabel('Distancia')
    ax1.grid(True, alpha=0.3, axis='y')
    
    # Subplot para clustering resultante
    ax2 = plt.subplot(2, 4, idx + 5)
    
    # Aplicar clustering jerárquico
    agg = AgglomerativeClustering(n_clusters=3, linkage=method)
    y_agg = agg.fit_predict(X_hier)
    
    # Visualizar
    scatter = ax2.scatter(X_hier[:, 0], X_hier[:, 1], c=y_agg, cmap='tab10', 
                         s=80, alpha=0.7, edgecolors='k', linewidths=1)
    ax2.set_title(f'Clusters (K=3)\n{method.capitalize()} Linkage', fontweight='bold', fontsize=11)
    ax2.set_xlabel('Feature 1')
    ax2.set_ylabel('Feature 2')
    ax2.grid(True, alpha=0.3)
    
    # Calcular métricas
    silhouette = silhouette_score(X_hier, y_agg)
    davies_bouldin = davies_bouldin_score(X_hier, y_agg)
    calinski = calinski_harabasz_score(X_hier, y_agg)
    
    # Añadir texto con métricas
    metrics_text = f'Silhouette: {silhouette:.3f}\nDavies-Bouldin: {davies_bouldin:.3f}'
    ax2.text(0.05, 0.95, metrics_text, transform=ax2.transAxes, 
            fontsize=9, verticalalignment='top',
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.show()

# Comparación detallada de métricas
print("\n📊 Comparación de Métodos de Enlace:")
print("=" * 80)
print(f"{'Método':<15} {'Silhouette':<15} {'Davies-Bouldin':<20} {'Calinski-Harabasz'}")
print("-" * 80)

for method in linkage_methods:
    agg = AgglomerativeClustering(n_clusters=3, linkage=method)
    y_pred = agg.fit_predict(X_hier)
    
    sil = silhouette_score(X_hier, y_pred)
    db = davies_bouldin_score(X_hier, y_pred)
    ch = calinski_harabasz_score(X_hier, y_pred)
    
    print(f"{method.capitalize():<15} {sil:<15.4f} {db:<20.4f} {ch:.2f}")

print("\n💡 Interpretación de Métricas:")
print("   • Silhouette Score: Mayor es mejor (rango [-1, 1])")
print("   • Davies-Bouldin: Menor es mejor (mide separación entre clusters)")
print("   • Calinski-Harabasz: Mayor es mejor (ratio varianza inter/intra cluster)")

## 4. DBSCAN (Density-Based Spatial Clustering)

### 🎯 Clustering Basado en Densidad

DBSCAN identifica clusters como **regiones de alta densidad** separadas por regiones de baja densidad, permitiendo descubrir clusters de forma arbitraria.

#### Conceptos Fundamentales:

- **ε-vecindad**: $N_\epsilon(x) = \{y \in \mathcal{D} : d(x,y) \leq \epsilon\}$
- **Core Point**: Punto con al menos `min_samples` vecinos en $N_\epsilon(x)$
- **Border Point**: No es core point pero está en vecindad de core point
- **Noise Point**: Ni core ni border point (outliers)

#### Algoritmo:
1. Para cada punto no visitado, determinar si es core point
2. Si es core point, iniciar nuevo cluster y añadir todos puntos density-reachable
3. Puntos no alcanzables se clasifican como **ruido**

### ✅ Propiedades Distintivas:
- **No requiere especificar K**: Número de clusters emerge de los datos
- **Forma Arbitraria**: No limitado a clusters convexos/esféricos
- **Robusto a Ruido**: Identifica y marca outliers explícitamente
- **Escalabilidad**: $O(n\log n)$ con estructuras de indexación espacial

### ⚠️ Consideraciones:
- Requiere selección cuidadosa de $\epsilon$ y `min_samples`
- Dificultad con clusters de densidades muy variables
- Sensible a escala de características (normalización recomendada)

In [ ]:
# Ejemplo Práctico: DBSCAN vs K-Means en Datos No Esféricos

print("🔬 Comparando DBSCAN con K-Means en datasets complejos...\n")

# Crear datasets con formas no convexas
np.random.seed(42)
datasets_complex = [
    (make_moons(n_samples=300, noise=0.05, random_state=42)[0], "Lunas"),
    (make_circles(n_samples=300, noise=0.05, factor=0.5, random_state=42)[0], "Círculos"),
    (make_blobs(n_samples=300, centers=[[0,0], [3,3], [0,3]], cluster_std=[0.4, 0.4, 0.4], random_state=42)[0], "Blobs Irregulares")
]

fig, axes = plt.subplots(len(datasets_complex), 3, figsize=(16, 12))

for idx, (X, name) in enumerate(datasets_complex):
    # 1. Datos originales
    axes[idx, 0].scatter(X[:, 0], X[:, 1], c='gray', s=50, alpha=0.6, edgecolors='k')
    axes[idx, 0].set_title(f'{name}\n(Datos Originales)', fontweight='bold', fontsize=11)
    axes[idx, 0].set_xlabel('Feature 1')
    axes[idx, 0].set_ylabel('Feature 2')
    axes[idx, 0].grid(True, alpha=0.3)
    
    # 2. K-Means
    kmeans = KMeans(n_clusters=2, init='k-means++', random_state=42, n_init=10)
    y_kmeans = kmeans.fit_predict(X)
    
    axes[idx, 1].scatter(X[:, 0], X[:, 1], c=y_kmeans, cmap='viridis', s=50, alpha=0.7, edgecolors='k')
    axes[idx, 1].scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1],
                        c='red', marker='X', s=300, edgecolors='black', linewidths=2, zorder=10)
    silhouette_km = silhouette_score(X, y_kmeans)
    axes[idx, 1].set_title(f'K-Means (K=2)\nSilhouette: {silhouette_km:.3f}', fontweight='bold', fontsize=11)
    axes[idx, 1].set_xlabel('Feature 1')
    axes[idx, 1].grid(True, alpha=0.3)
    
    # 3. DBSCAN
    # Normalizar datos para DBSCAN
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Ajustar parámetros según dataset
    if name == "Lunas":
        eps, min_samples = 0.15, 5
    elif name == "Círculos":
        eps, min_samples = 0.15, 5
    else:
        eps, min_samples = 0.3, 5
    
    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    y_dbscan = dbscan.fit_predict(X_scaled)
    
    # Visualizar resultados (distinguir ruido)
    unique_labels = set(y_dbscan)
    colors = plt.cm.Spectral(np.linspace(0, 1, len(unique_labels)))
    
    for k, col in zip(unique_labels, colors):
        if k == -1:
            # Ruido en negro
            col = [0, 0, 0, 1]
            marker = 'x'
            size = 30
            label = 'Ruido'
        else:
            marker = 'o'
            size = 50
            label = f'Cluster {k}'
        
        class_mask = (y_dbscan == k)
        axes[idx, 2].scatter(X[class_mask, 0], X[class_mask, 1], 
                           c=[col], marker=marker, s=size, alpha=0.7, 
                           edgecolors='k', linewidths=1, label=label)
    
    # Calcular silhouette solo para puntos no-ruido
    if len(set(y_dbscan)) > 1 and -1 not in y_dbscan:
        silhouette_db = silhouette_score(X, y_dbscan)
    else:
        mask_no_noise = y_dbscan != -1
        if np.sum(mask_no_noise) > 0 and len(set(y_dbscan[mask_no_noise])) > 1:
            silhouette_db = silhouette_score(X[mask_no_noise], y_dbscan[mask_no_noise])
        else:
            silhouette_db = 0.0
    
    n_clusters = len(set(y_dbscan)) - (1 if -1 in y_dbscan else 0)
    n_noise = list(y_dbscan).count(-1)
    
    axes[idx, 2].set_title(f'DBSCAN (ε={eps}, min={min_samples})\n'
                          f'Clusters: {n_clusters} | Ruido: {n_noise} | Sil: {silhouette_db:.3f}',
                          fontweight='bold', fontsize=11)
    axes[idx, 2].set_xlabel('Feature 1')
    axes[idx, 2].grid(True, alpha=0.3)
    axes[idx, 2].legend(fontsize=8, loc='best')

plt.tight_layout()
plt.show()

print("\n📊 Conclusiones:")
print("=" * 70)
print("✅ DBSCAN:")
print("   • Captura formas no convexas (lunas, círculos)")
print("   • Identifica ruido automáticamente")
print("   • No requiere especificar K")
print("\n⚠️  K-Means:")
print("   • Falla con geometrías complejas")
print("   • Asume clusters esféricos")
print("   • Todos los puntos asignados (sin detección de ruido)")
print("\n💡 Recomendación: DBSCAN para datos con formas irregulares y outliers")

## 5. Gaussian Mixture Models (GMM)

### 🎲 Clustering Probabilístico

GMM modela la distribución de datos como superposición de $K$ distribuciones gaussianas multivariadas:

$$p(x) = \sum_{k=1}^{K}\pi_k \mathcal{N}(x|\mu_k, \Sigma_k)$$

donde:
- $\pi_k$: pesos de mezcla ($\sum_k \pi_k = 1$)
- $\mu_k$: medias de cada componente
- $\Sigma_k$: matrices de covarianza

### Algoritmo Expectation-Maximization (EM)

#### E-step (Expectation):
Calcular responsabilidades (probabilidades posteriores):

$$\gamma_{ik} = \frac{\pi_k \mathcal{N}(x_i|\mu_k, \Sigma_k)}{\sum_{j=1}^{K}\pi_j \mathcal{N}(x_i|\mu_j, \Sigma_j)}$$

#### M-step (Maximization):
Actualizar parámetros:

$$\mu_k = \frac{\sum_i \gamma_{ik}x_i}{\sum_i \gamma_{ik}}, \quad \Sigma_k = \frac{\sum_i \gamma_{ik}(x_i - \mu_k)(x_i - \mu_k)^T}{\sum_i \gamma_{ik}}$$

### ✅ Ventajas:
- **Soft Clustering**: Pertenencia probabilística (no binaria)
- **Modelo Generativo**: Permite muestrear nuevos puntos
- **Flexibilidad**: Clusters elípticos mediante covarianzas
- **Fundamento Teórico**: Base estadística rigurosa

### ⚠️ Limitaciones:
- Sensible a inicialización (similar a K-Means)
- Convergencia a máximos locales
- Requiere especificar K
- Asume forma gaussiana

In [ ]:
# Ejemplo Práctico: GMM - Soft Clustering y Modelo Generativo

print("🎲 Aplicando Gaussian Mixture Models...\n")

# Generar datos con clusters elípticos
np.random.seed(42)
from sklearn.datasets import make_blobs
X_gmm, y_gmm_true = make_blobs(n_samples=300, centers=3, cluster_std=[0.5, 0.8, 0.6], random_state=42)

# Transformación para crear clusters elípticos
transformation = [[0.6, -0.6], [-0.4, 0.8]]
X_gmm = np.dot(X_gmm, transformation)

# Aplicar K-Means y GMM
kmeans_gmm = KMeans(n_clusters=3, random_state=42, n_init=10)
y_kmeans_gmm = kmeans_gmm.fit_predict(X_gmm)

gmm = GaussianMixture(n_components=3, covariance_type='full', random_state=42, n_init=10)
gmm.fit(X_gmm)
y_gmm_pred = gmm.predict(X_gmm)
y_gmm_proba = gmm.predict_proba(X_gmm)

# Visualización
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# 1. Ground Truth
axes[0, 0].scatter(X_gmm[:, 0], X_gmm[:, 1], c=y_gmm_true, cmap='viridis', s=50, alpha=0.7, edgecolors='k')
axes[0, 0].set_title('Ground Truth', fontweight='bold', fontsize=12)
axes[0, 0].set_xlabel('Feature 1')
axes[0, 0].set_ylabel('Feature 2')
axes[0, 0].grid(True, alpha=0.3)

# 2. K-Means (Hard Clustering)
axes[0, 1].scatter(X_gmm[:, 0], X_gmm[:, 1], c=y_kmeans_gmm, cmap='plasma', s=50, alpha=0.7, edgecolors='k')
axes[0, 1].scatter(kmeans_gmm.cluster_centers_[:, 0], kmeans_gmm.cluster_centers_[:, 1],
                  c='red', marker='X', s=300, edgecolors='black', linewidths=2, zorder=10)
silhouette_km_gmm = silhouette_score(X_gmm, y_kmeans_gmm)
axes[0, 1].set_title(f'K-Means (Hard Clustering)\nSilhouette: {silhouette_km_gmm:.3f}', fontweight='bold', fontsize=12)
axes[0, 1].set_xlabel('Feature 1')
axes[0, 1].grid(True, alpha=0.3)

# 3. GMM (Hard Clustering)
axes[0, 2].scatter(X_gmm[:, 0], X_gmm[:, 1], c=y_gmm_pred, cmap='Set1', s=50, alpha=0.7, edgecolors='k')
axes[0, 2].scatter(gmm.means_[:, 0], gmm.means_[:, 1],
                  c='black', marker='D', s=300, edgecolors='yellow', linewidths=2, zorder=10, label='Medias')
silhouette_gmm = silhouette_score(X_gmm, y_gmm_pred)
axes[0, 2].set_title(f'GMM (Hard Clustering)\nSilhouette: {silhouette_gmm:.3f}', fontweight='bold', fontsize=12)
axes[0, 2].set_xlabel('Feature 1')
axes[0, 2].legend()
axes[0, 2].grid(True, alpha=0.3)

# 4. GMM - Probabilidades de Pertenencia (Cluster 0)
scatter = axes[1, 0].scatter(X_gmm[:, 0], X_gmm[:, 1], c=y_gmm_proba[:, 0], 
                             cmap='RdYlGn', s=60, alpha=0.8, edgecolors='k', vmin=0, vmax=1)
plt.colorbar(scatter, ax=axes[1, 0], label='P(Cluster 0)')
axes[1, 0].set_title('GMM: Probabilidad Cluster 0', fontweight='bold', fontsize=12)
axes[1, 0].set_xlabel('Feature 1')
axes[1, 0].set_ylabel('Feature 2')
axes[1, 0].grid(True, alpha=0.3)

# 5. GMM - Muestras Generadas (Modelo Generativo)
X_generated, y_generated = gmm.sample(300)
axes[1, 1].scatter(X_generated[:, 0], X_generated[:, 1], c=y_generated, 
                  cmap='Set1', s=50, alpha=0.6, edgecolors='k', marker='s')
axes[1, 1].scatter(gmm.means_[:, 0], gmm.means_[:, 1],
                  c='black', marker='D', s=300, edgecolors='yellow', linewidths=2, zorder=10)
axes[1, 1].set_title('Datos Generados por GMM\n(Modelo Generativo)', fontweight='bold', fontsize=12)
axes[1, 1].set_xlabel('Feature 1')
axes[1, 1].grid(True, alpha=0.3)

# 6. Elipses de covarianza GMM
from matplotlib.patches import Ellipse

axes[1, 2].scatter(X_gmm[:, 0], X_gmm[:, 1], c=y_gmm_pred, cmap='Set1', s=30, alpha=0.5, edgecolors='k')
axes[1, 2].scatter(gmm.means_[:, 0], gmm.means_[:, 1],
                  c='black', marker='D', s=300, edgecolors='yellow', linewidths=2, zorder=10)

# Dibujar elipses de covarianza (2 desviaciones estándar)
for i in range(3):
    covariance = gmm.covariances_[i]
    v, w = np.linalg.eigh(covariance)
    v = 2.0 * np.sqrt(2.0) * np.sqrt(v)  # 2 std deviations
    u = w[0] / np.linalg.norm(w[0])
    angle = np.arctan2(u[1], u[0])
    angle = 180.0 * angle / np.pi
    
    ell = Ellipse(gmm.means_[i], v[0], v[1], angle=180.0 + angle, 
                  edgecolor=f'C{i}', facecolor='none', linewidth=3, linestyle='--')
    axes[1, 2].add_patch(ell)

axes[1, 2].set_title('GMM: Elipses de Covarianza\n(2σ)', fontweight='bold', fontsize=12)
axes[1, 2].set_xlabel('Feature 1')
axes[1, 2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 Análisis Detallado:")
print("=" * 70)
print(f"GMM - BIC Score: {gmm.bic(X_gmm):.2f} (menor es mejor)")
print(f"GMM - AIC Score: {gmm.aic(X_gmm):.2f} (menor es mejor)")
print(f"GMM - Log-Likelihood: {gmm.score(X_gmm):.2f}")

print("\n📈 Pesos de las Componentes:")
for i, weight in enumerate(gmm.weights_):
    print(f"   Cluster {i}: {weight:.3f} ({weight*100:.1f}%)")

print("\n💡 Ventajas de GMM sobre K-Means:")
print("   ✅ Soft clustering: probabilidades en lugar de asignaciones binarias")
print("   ✅ Modelo generativo: puede generar nuevas muestras")
print("   ✅ Captura forma elíptica mediante covarianzas completas")
print("   ✅ Fundamento probabilístico riguroso")

## 6. Reducción de Dimensionalidad: PCA

### 📉 Principal Component Analysis

PCA identifica **direcciones de máxima varianza** en los datos, proyectando observaciones a un subespacio de menor dimensión.

#### Fundamento Matemático

Dada matriz de datos centrados $X \in \mathbb{R}^{n \times d}$, PCA busca proyección ortogonal $W \in \mathbb{R}^{d \times k}$ que maximiza varianza:

$$\max_W \text{tr}(W^T\Sigma W) \quad \text{sujeto a } W^TW = I_k$$

donde $\Sigma = \frac{1}{n}X^TX$ es la matriz de covarianza muestral.

#### Solución:
Los **vectores propios** de $\Sigma$ correspondientes a los $k$ mayores valores propios forman las columnas de $W$.

Equivalentemente: $X = UDV^T$ (Singular Value Decomposition)

### Propiedades Clave:
- Transformación lineal óptima bajo criterio de reconstrucción de mínimos cuadrados
- Componentes principales son **ortogonales** (no correlacionados)
- Varianza explicada por PC $k$: $\lambda_k / \sum_i \lambda_i$

### ✅ Aplicaciones:
- Compresión de datos
- Visualización mediante proyección a 2D/3D
- Eliminación de multicolinealidad
- Pre-procesamiento para aceleración de algoritmos
- Reducción de ruido

### ⚠️ Limitaciones:
- Asume **linealidad** de relaciones
- Sensible a escala (requiere normalización)
- Componentes pueden ser difíciles de interpretar
- No preserva distancias locales necesariamente

In [ ]:
# Ejemplo Práctico: PCA - Visualización y Reducción de Dimensionalidad

print("📉 Aplicando PCA a Digits Dataset (64 dimensiones → 2D/3D)...\n")

# Cargar datos de dígitos manuscritos (8x8 píxeles = 64 características)
digits = load_digits()
X_digits = digits.data
y_digits = digits.target

print(f"Dataset original: {X_digits.shape[0]} muestras, {X_digits.shape[1]} características")
print(f"Clases: {len(np.unique(y_digits))} dígitos (0-9)")

# Normalizar datos
scaler_pca = StandardScaler()
X_digits_scaled = scaler_pca.fit_transform(X_digits)

# Aplicar PCA con todos los componentes para análisis de varianza
pca_full = PCA()
pca_full.fit(X_digits_scaled)

# Crear visualizaciones
fig = plt.figure(figsize=(18, 12))

# 1. Varianza explicada por componente
ax1 = plt.subplot(2, 3, 1)
ax1.bar(range(1, 21), pca_full.explained_variance_ratio_[:20] * 100, alpha=0.7, color='steelblue')
ax1.set_xlabel('Componente Principal')
ax1.set_ylabel('Varianza Explicada (%)')
ax1.set_title('Varianza Explicada por PC\n(Top 20 componentes)', fontweight='bold', fontsize=11)
ax1.grid(True, alpha=0.3, axis='y')

# 2. Varianza acumulada
ax2 = plt.subplot(2, 3, 2)
cumsum_variance = np.cumsum(pca_full.explained_variance_ratio_)
ax2.plot(range(1, len(cumsum_variance) + 1), cumsum_variance * 100, 'o-', linewidth=2, markersize=4)
ax2.axhline(y=95, color='r', linestyle='--', linewidth=2, label='95% varianza')
ax2.axhline(y=90, color='orange', linestyle='--', linewidth=2, label='90% varianza')
ax2.set_xlabel('Número de Componentes')
ax2.set_ylabel('Varianza Acumulada (%)')
ax2.set_title('Varianza Explicada Acumulada', fontweight='bold', fontsize=11)
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.set_xlim([0, 50])

# Encontrar número de componentes para 95% varianza
n_components_95 = np.argmax(cumsum_variance >= 0.95) + 1
print(f"\n📊 Componentes necesarios para 95% varianza: {n_components_95}/{X_digits.shape[1]}")

# 3. Primeros 6 componentes principales visualizados
ax3 = plt.subplot(2, 3, 3)
for i in range(6):
    ax_small = plt.subplot(2, 6, i + 7)
    component_image = pca_full.components_[i].reshape(8, 8)
    ax_small.imshow(component_image, cmap='RdBu_r', aspect='auto')
    ax_small.set_title(f'PC{i+1}\n{pca_full.explained_variance_ratio_[i]*100:.1f}%', fontsize=9)
    ax_small.axis('off')

# 4. PCA 2D - Visualización
pca_2d = PCA(n_components=2)
X_pca_2d = pca_2d.fit_transform(X_digits_scaled)

ax4 = plt.subplot(2, 3, 4)
scatter = ax4.scatter(X_pca_2d[:, 0], X_pca_2d[:, 1], c=y_digits, cmap='tab10', 
                     s=20, alpha=0.6, edgecolors='none')
ax4.set_xlabel(f'PC1 ({pca_2d.explained_variance_ratio_[0]*100:.1f}% var)')
ax4.set_ylabel(f'PC2 ({pca_2d.explained_variance_ratio_[1]*100:.1f}% var)')
ax4.set_title(f'PCA 2D Projection\nVarianza total: {sum(pca_2d.explained_variance_ratio_)*100:.1f}%', 
             fontweight='bold', fontsize=11)
plt.colorbar(scatter, ax=ax4, label='Dígito', ticks=range(10))
ax4.grid(True, alpha=0.3)

# 5. Reconstrucción con diferentes números de componentes
ax5 = plt.subplot(2, 3, 5)

# Seleccionar una muestra (dígito '5')
sample_idx = np.where(y_digits == 5)[0][0]
original_digit = X_digits[sample_idx].reshape(8, 8)

# Reconstruir con diferentes números de componentes
n_components_list = [2, 5, 10, 20, 64]
reconstructions = []

for n_comp in n_components_list:
    if n_comp <= 64:
        pca_temp = PCA(n_components=n_comp)
        pca_temp.fit(X_digits_scaled)
        transformed = pca_temp.transform(X_digits_scaled[sample_idx:sample_idx+1])
        reconstructed_scaled = pca_temp.inverse_transform(transformed)
        reconstructed = scaler_pca.inverse_transform(reconstructed_scaled)
        reconstructions.append(reconstructed.reshape(8, 8))
    else:
        reconstructions.append(original_digit)

# Mostrar reconstrucciones
for idx, (recon, n_comp) in enumerate(zip(reconstructions, n_components_list)):
    ax_recon = plt.subplot(2, 6, idx + 1)
    ax_recon.imshow(recon, cmap='gray', aspect='auto')
    
    # Calcular MSE
    mse = np.mean((original_digit - recon) ** 2)
    ax_recon.set_title(f'{n_comp} PCs\nMSE: {mse:.2f}', fontsize=9)
    ax_recon.axis('off')

# 6. PCA aplicado a clustering
ax6 = plt.subplot(2, 3, 6)

# Aplicar K-Means en espacio PCA reducido
kmeans_pca = KMeans(n_clusters=10, random_state=42, n_init=10)
y_kmeans_pca = kmeans_pca.fit_predict(X_pca_2d)

scatter2 = ax6.scatter(X_pca_2d[:, 0], X_pca_2d[:, 1], c=y_kmeans_pca, cmap='tab10', 
                      s=20, alpha=0.6, edgecolors='none')
ax6.scatter(kmeans_pca.cluster_centers_[:, 0], kmeans_pca.cluster_centers_[:, 1],
           c='red', marker='X', s=200, edgecolors='black', linewidths=2, zorder=10)
ax6.set_xlabel('PC1')
ax6.set_ylabel('PC2')
ax6.set_title('K-Means en Espacio PCA\n(10 clusters)', fontweight='bold', fontsize=11)
ax6.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📈 Resumen PCA:")
print("=" * 70)
print(f"   • Varianza explicada por PC1: {pca_2d.explained_variance_ratio_[0]*100:.2f}%")
print(f"   • Varianza explicada por PC2: {pca_2d.explained_variance_ratio_[1]*100:.2f}%")
print(f"   • Varianza total (2 PCs): {sum(pca_2d.explained_variance_ratio_)*100:.2f}%")
print(f"   • Componentes para 95% varianza: {n_components_95}/64")
print(f"   • Reducción dimensional: 64D → 2D ({(1-2/64)*100:.1f}% reducción)")

print("\n💡 Aplicaciones Prácticas:")
print("   ✅ Visualización de datos de alta dimensión")
print("   ✅ Pre-procesamiento para acelerar algoritmos (clustering, clasificación)")
print("   ✅ Compresión de datos (trade-off entre compresión y pérdida)")
print("   ✅ Eliminación de ruido (reconstrucción con componentes principales)")
print("   ✅ Detección de multicolinealidad en features")

## 7. t-SNE: Visualización No Lineal

### 🎨 t-Distributed Stochastic Neighbor Embedding

t-SNE es una técnica **no lineal** de reducción de dimensionalidad especialmente efectiva para **visualización**, preservando estructura local de datos.

#### Fundamento Matemático

**En espacio original**, convierte distancias en probabilidades condicionales:

$$p_{j|i} = \frac{\exp(-\|x_i - x_j\|^2 / 2\sigma_i^2)}{\sum_{k \neq i}\exp(-\|x_i - x_k\|^2 / 2\sigma_i^2)}$$

**En espacio reducido**, usa distribución t de Student (cola pesada):

$$q_{ij} = \frac{(1 + \|y_i - y_j\|^2)^{-1}}{\sum_{k \neq l}(1 + \|y_k - y_l\|^2)^{-1}}$$

**Optimización**: Minimiza divergencia KL entre $P$ y $Q$:

$$\text{KL}(P\|Q) = \sum_{i \neq j} p_{ij}\log\frac{p_{ij}}{q_{ij}}$$

### Parámetros Clave:

- **perplexity**: Controla número efectivo de vecinos cercanos (típico: 5-50)
  - Bajo: énfasis en estructura local
  - Alto: énfasis en estructura global

- **learning_rate**: Tasa de aprendizaje (típico: 10-1000)

- **n_iter**: Iteraciones de optimización (mínimo: 250, recomendado: 1000+)

### ✅ Ventajas:
- Preserva estructura local (vecindarios)
- Visualización efectiva de datos complejos
- Revela clusters no lineales

### ⚠️ Limitaciones:
- **No determinístico** (diferentes ejecuciones dan resultados diferentes)
- **Computacionalmente costoso**: $O(n^2)$ (mitigado con Barnes-Hut)
- **Solo para visualización** (no para reducción dimensional en pipeline)
- Distancias globales no preservadas
- Sensible a parámetros (perplexity)

In [ ]:
# Ejemplo Práctico: PCA vs t-SNE - Comparación en Datos Complejos

print("🎨 Comparando PCA (lineal) vs t-SNE (no lineal) en Digits Dataset...\n")

# Usar submuestra para t-SNE (más rápido)
np.random.seed(42)
n_samples = 1000
indices = np.random.choice(len(X_digits), n_samples, replace=False)
X_sample = X_digits_scaled[indices]
y_sample = y_digits[indices]

print(f"Usando {n_samples} muestras para comparación...")

# Aplicar PCA (ya calculado anteriormente, recalcular para subset)
pca_comp = PCA(n_components=2)
X_pca_comp = pca_comp.fit_transform(X_sample)

# Aplicar t-SNE con diferentes perplexities
perplexities = [5, 30, 50]
tsne_results = []

print("\nAplicando t-SNE con diferentes perplexities...")
for perp in perplexities:
    print(f"   • Perplexity {perp}...", end='')
    tsne = TSNE(n_components=2, perplexity=perp, random_state=42, n_iter=1000, 
                learning_rate='auto', init='pca')
    X_tsne = tsne.fit_transform(X_sample)
    tsne_results.append((perp, X_tsne))
    print(" ✓")

# Visualización comparativa
fig, axes = plt.subplots(2, 2, figsize=(16, 14))

# 1. PCA
scatter1 = axes[0, 0].scatter(X_pca_comp[:, 0], X_pca_comp[:, 1], c=y_sample, 
                             cmap='tab10', s=30, alpha=0.7, edgecolors='k', linewidths=0.5)
axes[0, 0].set_title(f'PCA (Lineal)\nVarianza: {sum(pca_comp.explained_variance_ratio_)*100:.1f}%', 
                    fontweight='bold', fontsize=13)
axes[0, 0].set_xlabel('PC1')
axes[0, 0].set_ylabel('PC2')
axes[0, 0].grid(True, alpha=0.3)
plt.colorbar(scatter1, ax=axes[0, 0], label='Dígito', ticks=range(10))

# 2-4. t-SNE con diferentes perplexities
for idx, (perp, X_tsne) in enumerate(tsne_results):
    row = (idx + 1) // 2
    col = (idx + 1) % 2
    
    scatter = axes[row, col].scatter(X_tsne[:, 0], X_tsne[:, 1], c=y_sample, 
                                    cmap='tab10', s=30, alpha=0.7, edgecolors='k', linewidths=0.5)
    axes[row, col].set_title(f't-SNE (No Lineal)\nPerplexity: {perp}', 
                            fontweight='bold', fontsize=13)
    axes[row, col].set_xlabel('t-SNE Dim 1')
    axes[row, col].set_ylabel('t-SNE Dim 2')
    axes[row, col].grid(True, alpha=0.3)
    plt.colorbar(scatter, ax=axes[row, col], label='Dígito', ticks=range(10))

plt.tight_layout()
plt.show()

# Análisis cuantitativo: clustering en espacio reducido
print("\n📊 Evaluación de Clustering en Espacio Reducido:")
print("=" * 80)
print(f"{'Método':<20} {'Silhouette':<15} {'Davies-Bouldin':<20} {'Calinski-Harabasz'}")
print("-" * 80)

# PCA + K-Means
kmeans_pca_eval = KMeans(n_clusters=10, random_state=42, n_init=10)
y_pred_pca = kmeans_pca_eval.fit_predict(X_pca_comp)
sil_pca = silhouette_score(X_pca_comp, y_pred_pca)
db_pca = davies_bouldin_score(X_pca_comp, y_pred_pca)
ch_pca = calinski_harabasz_score(X_pca_comp, y_pred_pca)
print(f"{'PCA':<20} {sil_pca:<15.4f} {db_pca:<20.4f} {ch_pca:.2f}")

# t-SNE + K-Means (usando perplexity=30)
X_tsne_30 = [x for p, x in tsne_results if p == 30][0]
kmeans_tsne = KMeans(n_clusters=10, random_state=42, n_init=10)
y_pred_tsne = kmeans_tsne.fit_predict(X_tsne_30)
sil_tsne = silhouette_score(X_tsne_30, y_pred_tsne)
db_tsne = davies_bouldin_score(X_tsne_30, y_pred_tsne)
ch_tsne = calinski_harabasz_score(X_tsne_30, y_pred_tsne)
print(f"{'t-SNE (perp=30)':<20} {sil_tsne:<15.4f} {db_tsne:<20.4f} {ch_tsne:.2f}")

print("\n💡 Conclusiones:")
print("=" * 70)
print("✅ PCA:")
print("   • Rápido y determinístico")
print("   • Preserva varianza global")
print("   • Útil para pre-procesamiento y compresión")
print("   • Limitado para estructuras no lineales")

print("\n✅ t-SNE:")
print("   • Visualización superior de estructuras complejas")
print("   • Preserva relaciones locales (vecindarios)")
print("   • Revela clusters no lineales claramente")
print("   • Más lento, no determinístico")
print("   • Sensible a perplexity (experimentar con valores 5-50)")

print("\n🎯 Recomendaciones:")
print("   • Usar PCA para análisis exploratorio rápido y reducción dimensional")
print("   • Usar t-SNE para visualización final de alta calidad")
print("   • Combinar: PCA primero (64D→50D), luego t-SNE (50D→2D) para acelerar")

## 8. Detección de Anomalías

### 🚨 Identificación de Outliers

La detección de anomalías identifica observaciones que se desvían significativamente de patrones normales.

#### Isolation Forest

**Principio**: Las anomalías son "fáciles de aislar" (requieren menos particiones en árboles aleatorios).

**Anomaly Score**: Basado en longitud promedio de camino:

$$s(x, n) = 2^{-\frac{E(h(x))}{c(n)}}$$

donde $h(x)$ es profundidad de aislamiento y $c(n)$ es profundidad promedio esperada.

- Score ≈ 1: Anomalía
- Score ≈ 0.5: Normal
- Score < 0.5: Normal con alta confianza

#### Local Outlier Factor (LOF)

**Principio**: Compara densidad local de un punto con densidad de sus vecinos.

$$\text{LOF}_k(x) = \frac{\sum_{o \in N_k(x)} \frac{\text{lrd}(o)}{\text{lrd}(x)}}{|N_k(x)|}$$

donde $\text{lrd}$ es local reachability density.

- LOF ≈ 1: Normal (densidad similar a vecinos)
- LOF >> 1: Anomalía (densidad menor que vecinos)

### ✅ Ventajas:
- **Isolation Forest**: Eficiente, escalable, maneja alta dimensionalidad
- **LOF**: Detecta anomalías locales, flexible con diferentes densidades

### ⚠️ Consideraciones:
- Requiere especificar tasa de contaminación esperada
- Sensible a parámetros (n_neighbors en LOF)
- Evaluación difícil sin ground truth

In [ ]:
# Ejemplo Práctico: Detección de Anomalías - Isolation Forest vs LOF

print("🚨 Aplicando algoritmos de detección de anomalías...\n")

# Generar dataset con outliers
np.random.seed(42)
n_samples = 300
n_outliers = 30

# Datos normales
X_normal = np.random.randn(n_samples, 2) * 0.5 + np.array([0, 0])

# Outliers
X_outliers = np.random.uniform(low=-4, high=4, size=(n_outliers, 2))

# Combinar
X_anomaly = np.vstack([X_normal, X_outliers])
y_true = np.array([0] * n_samples + [1] * n_outliers)  # 0=normal, 1=outlier

print(f"Dataset: {len(X_anomaly)} puntos ({n_samples} normales + {n_outliers} outliers)")

# Aplicar Isolation Forest
contamination_rate = n_outliers / len(X_anomaly)
iso_forest = IsolationForest(contamination=contamination_rate, random_state=42, n_estimators=100)
y_pred_iso = iso_forest.fit_predict(X_anomaly)
y_pred_iso = np.where(y_pred_iso == -1, 1, 0)  # Convertir -1/1 a 1/0

# Obtener anomaly scores
anomaly_scores_iso = -iso_forest.score_samples(X_anomaly)  # Negar para que mayor = más anómalo

# Aplicar Local Outlier Factor
lof = LocalOutlierFactor(n_neighbors=20, contamination=contamination_rate)
y_pred_lof = lof.fit_predict(X_anomaly)
y_pred_lof = np.where(y_pred_lof == -1, 1, 0)

# Obtener LOF scores
lof_scores = -lof.negative_outlier_factor_

# Visualización
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# 1. Ground Truth
axes[0, 0].scatter(X_anomaly[y_true==0, 0], X_anomaly[y_true==0, 1], 
                  c='blue', label='Normal', s=50, alpha=0.6, edgecolors='k')
axes[0, 0].scatter(X_anomaly[y_true==1, 0], X_anomaly[y_true==1, 1], 
                  c='red', label='Outlier', s=100, alpha=0.8, marker='X', edgecolors='black', linewidths=2)
axes[0, 0].set_title('Ground Truth', fontweight='bold', fontsize=12)
axes[0, 0].set_xlabel('Feature 1')
axes[0, 0].set_ylabel('Feature 2')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 2. Isolation Forest - Predicciones
axes[0, 1].scatter(X_anomaly[y_pred_iso==0, 0], X_anomaly[y_pred_iso==0, 1], 
                  c='blue', label='Normal', s=50, alpha=0.6, edgecolors='k')
axes[0, 1].scatter(X_anomaly[y_pred_iso==1, 0], X_anomaly[y_pred_iso==1, 1], 
                  c='red', label='Outlier', s=100, alpha=0.8, marker='X', edgecolors='black', linewidths=2)

# Calcular métricas
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
precision_iso = precision_score(y_true, y_pred_iso)
recall_iso = recall_score(y_true, y_pred_iso)
f1_iso = f1_score(y_true, y_pred_iso)

axes[0, 1].set_title(f'Isolation Forest\nPrec: {precision_iso:.3f} | Rec: {recall_iso:.3f} | F1: {f1_iso:.3f}', 
                    fontweight='bold', fontsize=12)
axes[0, 1].set_xlabel('Feature 1')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# 3. LOF - Predicciones
axes[0, 2].scatter(X_anomaly[y_pred_lof==0, 0], X_anomaly[y_pred_lof==0, 1], 
                  c='blue', label='Normal', s=50, alpha=0.6, edgecolors='k')
axes[0, 2].scatter(X_anomaly[y_pred_lof==1, 0], X_anomaly[y_pred_lof==1, 1], 
                  c='red', label='Outlier', s=100, alpha=0.8, marker='X', edgecolors='black', linewidths=2)

precision_lof = precision_score(y_true, y_pred_lof)
recall_lof = recall_score(y_true, y_pred_lof)
f1_lof = f1_score(y_true, y_pred_lof)

axes[0, 2].set_title(f'Local Outlier Factor\nPrec: {precision_lof:.3f} | Rec: {recall_lof:.3f} | F1: {f1_lof:.3f}', 
                    fontweight='bold', fontsize=12)
axes[0, 2].set_xlabel('Feature 1')
axes[0, 2].legend()
axes[0, 2].grid(True, alpha=0.3)

# 4. Isolation Forest - Anomaly Scores
scatter_iso_scores = axes[1, 0].scatter(X_anomaly[:, 0], X_anomaly[:, 1], 
                                       c=anomaly_scores_iso, cmap='RdYlGn_r', 
                                       s=60, alpha=0.7, edgecolors='k', linewidths=0.5)
plt.colorbar(scatter_iso_scores, ax=axes[1, 0], label='Anomaly Score')
axes[1, 0].set_title('Isolation Forest\nAnomaly Scores', fontweight='bold', fontsize=12)
axes[1, 0].set_xlabel('Feature 1')
axes[1, 0].set_ylabel('Feature 2')
axes[1, 0].grid(True, alpha=0.3)

# 5. LOF - Scores
scatter_lof_scores = axes[1, 1].scatter(X_anomaly[:, 0], X_anomaly[:, 1], 
                                       c=lof_scores, cmap='RdYlGn_r', 
                                       s=60, alpha=0.7, edgecolors='k', linewidths=0.5)
plt.colorbar(scatter_lof_scores, ax=axes[1, 1], label='LOF Score')
axes[1, 1].set_title('Local Outlier Factor\nLOF Scores', fontweight='bold', fontsize=12)
axes[1, 1].set_xlabel('Feature 1')
axes[1, 1].grid(True, alpha=0.3)

# 6. Confusion Matrices
cm_iso = confusion_matrix(y_true, y_pred_iso)
cm_lof = confusion_matrix(y_true, y_pred_lof)

ax_cm1 = plt.subplot(2, 6, 11)
sns.heatmap(cm_iso, annot=True, fmt='d', cmap='Blues', ax=ax_cm1,
           xticklabels=['Normal', 'Outlier'], yticklabels=['Normal', 'Outlier'])
ax_cm1.set_title('Isolation Forest\nConfusion Matrix', fontsize=10, fontweight='bold')
ax_cm1.set_ylabel('True')
ax_cm1.set_xlabel('Predicted')

ax_cm2 = plt.subplot(2, 6, 12)
sns.heatmap(cm_lof, annot=True, fmt='d', cmap='Greens', ax=ax_cm2,
           xticklabels=['Normal', 'Outlier'], yticklabels=['Normal', 'Outlier'])
ax_cm2.set_title('LOF\nConfusion Matrix', fontsize=10, fontweight='bold')
ax_cm2.set_ylabel('True')
ax_cm2.set_xlabel('Predicted')

plt.tight_layout()
plt.show()

# Análisis detallado
print("\n📊 Comparación de Algoritmos:")
print("=" * 70)
print(f"{'Algoritmo':<20} {'Precision':<12} {'Recall':<12} {'F1-Score':<12}")
print("-" * 70)
print(f"{'Isolation Forest':<20} {precision_iso:<12.4f} {recall_iso:<12.4f} {f1_iso:<12.4f}")
print(f"{'LOF':<20} {precision_lof:<12.4f} {recall_lof:<12.4f} {f1_lof:<12.4f}")

print("\n💡 Interpretación:")
print("   • Precision: De los puntos marcados como outliers, ¿cuántos lo son realmente?")
print("   • Recall: De todos los outliers reales, ¿cuántos se detectaron?")
print("   • F1-Score: Media armónica de Precision y Recall")

print("\n🎯 Recomendaciones de Uso:")
print("=" * 70)
print("✅ Isolation Forest:")
print("   • Escalable a grandes datasets")
print("   • Funciona bien en alta dimensionalidad")
print("   • Más rápido que LOF")
print("   • Ideal para outliers globales")

print("\n✅ Local Outlier Factor:")
print("   • Detecta anomalías locales (outliers contextuales)")
print("   • Maneja clusters de diferentes densidades")
print("   • Más sensible a patrones locales")
print("   • Requiere ajuste de n_neighbors")

## 9. Caso Práctico: Wine Dataset

Aplicaremos técnicas de aprendizaje no supervisado al Wine Dataset para descubrir patrones sin usar las etiquetas.

In [ ]:
# Caso Práctico Completo: Wine Dataset - Pipeline de Análisis No Supervisado

print("🍷 Análisis No Supervisado del Wine Dataset\n")

# Cargar datos
wine = load_wine()
X_wine = wine.data
y_wine_true = wine.target  # Solo para evaluación, no se usa en entrenamiento

# Crear DataFrame
df_wine = pd.DataFrame(X_wine, columns=wine.feature_names)
df_wine['cultivar'] = y_wine_true

print(f"Dataset: {X_wine.shape[0]} muestras, {X_wine.shape[1]} características")
print(f"Clases reales: {len(np.unique(y_wine_true))} cultivares de vino\n")

print("📊 Primeras 5 filas:")
print(df_wine.head())

# Normalizar datos
scaler_wine = StandardScaler()
X_wine_scaled = scaler_wine.fit_transform(X_wine)

# Pipeline de análisis no supervisado
fig = plt.figure(figsize=(20, 14))

# 1. Clustering: Comparación de algoritmos
print("\n🔬 1. CLUSTERING - Comparación de algoritmos...")

clustering_algos = {
    'K-Means': KMeans(n_clusters=3, random_state=42, n_init=10),
    'Hierarchical': AgglomerativeClustering(n_clusters=3, linkage='ward'),
    'GMM': GaussianMixture(n_components=3, random_state=42, n_init=10)
}

clustering_results = {}

for name, algo in clustering_algos.items():
    if name == 'GMM':
        algo.fit(X_wine_scaled)
        y_pred = algo.predict(X_wine_scaled)
    else:
        y_pred = algo.fit_predict(X_wine_scaled)
    
    clustering_results[name] = {
        'labels': y_pred,
        'silhouette': silhouette_score(X_wine_scaled, y_pred),
        'davies_bouldin': davies_bouldin_score(X_wine_scaled, y_pred),
        'calinski': calinski_harabasz_score(X_wine_scaled, y_pred)
    }

# Tabla de resultados clustering
print("\n📈 Métricas de Clustering:")
print("=" * 80)
print(f"{'Algoritmo':<15} {'Silhouette':<15} {'Davies-Bouldin':<20} {'Calinski-Harabasz'}")
print("-" * 80)
for name, results in clustering_results.items():
    print(f"{name:<15} {results['silhouette']:<15.4f} {results['davies_bouldin']:<20.4f} {results['calinski']:.2f}")

# 2. PCA - Reducción de dimensionalidad
print("\n📉 2. REDUCCIÓN DE DIMENSIONALIDAD (PCA)...")

pca_wine = PCA()
pca_wine.fit(X_wine_scaled)

# Encontrar componentes para 95% varianza
cumsum_var_wine = np.cumsum(pca_wine.explained_variance_ratio_)
n_comp_95_wine = np.argmax(cumsum_var_wine >= 0.95) + 1

print(f"   • Componentes para 95% varianza: {n_comp_95_wine}/{X_wine.shape[1]}")

# Aplicar PCA con 2 componentes para visualización
pca_2d_wine = PCA(n_components=2)
X_wine_pca = pca_2d_wine.fit_transform(X_wine_scaled)

print(f"   • Varianza explicada (2 PCs): {sum(pca_2d_wine.explained_variance_ratio_)*100:.2f}%")

# 3. Visualizaciones
# Subplot 1: PCA - Ground Truth
ax1 = plt.subplot(3, 3, 1)
scatter1 = ax1.scatter(X_wine_pca[:, 0], X_wine_pca[:, 1], c=y_wine_true, 
                      cmap='viridis', s=80, alpha=0.7, edgecolors='k')
ax1.set_title('PCA - Ground Truth\n(Cultivares Reales)', fontweight='bold', fontsize=11)
ax1.set_xlabel(f'PC1 ({pca_2d_wine.explained_variance_ratio_[0]*100:.1f}%)')
ax1.set_ylabel(f'PC2 ({pca_2d_wine.explained_variance_ratio_[1]*100:.1f}%)')
plt.colorbar(scatter1, ax=ax1, label='Cultivar')
ax1.grid(True, alpha=0.3)

# Subplots 2-4: Clustering en espacio PCA
for idx, (name, results) in enumerate(clustering_results.items()):
    ax = plt.subplot(3, 3, idx + 2)
    scatter = ax.scatter(X_wine_pca[:, 0], X_wine_pca[:, 1], c=results['labels'], 
                        cmap='plasma', s=80, alpha=0.7, edgecolors='k')
    ax.set_title(f'{name} en PCA\nSil: {results["silhouette"]:.3f}', 
                fontweight='bold', fontsize=11)
    ax.set_xlabel('PC1')
    ax.set_ylabel('PC2')
    plt.colorbar(scatter, ax=ax, label='Cluster')
    ax.grid(True, alpha=0.3)

# Subplot 5: Varianza explicada PCA
ax5 = plt.subplot(3, 3, 5)
ax5.bar(range(1, len(pca_wine.explained_variance_ratio_) + 1), 
       pca_wine.explained_variance_ratio_ * 100, alpha=0.7, color='steelblue')
ax5.axhline(y=pca_wine.explained_variance_ratio_[0]*100, color='r', 
           linestyle='--', alpha=0.7, label=f'PC1: {pca_wine.explained_variance_ratio_[0]*100:.1f}%')
ax5.set_xlabel('Componente Principal')
ax5.set_ylabel('Varianza Explicada (%)')
ax5.set_title('Scree Plot - PCA', fontweight='bold', fontsize=11)
ax5.legend()
ax5.grid(True, alpha=0.3, axis='y')

# Subplot 6: Feature importance en PC1 y PC2
ax6 = plt.subplot(3, 3, 6)
features_to_show = 8  # Top 8 features
pc1_importance = np.abs(pca_2d_wine.components_[0])
pc2_importance = np.abs(pca_2d_wine.components_[1])

top_features_pc1 = np.argsort(pc1_importance)[-features_to_show:]
x_pos = np.arange(features_to_show)
width = 0.35

ax6.barh(x_pos, pc1_importance[top_features_pc1], width, label='PC1', alpha=0.8)
ax6.barh(x_pos + width, pc2_importance[top_features_pc1], width, label='PC2', alpha=0.8)
ax6.set_yticks(x_pos + width / 2)
ax6.set_yticklabels([wine.feature_names[i][:20] for i in top_features_pc1], fontsize=8)
ax6.set_xlabel('Importancia Absoluta')
ax6.set_title('Top Features en PC1 y PC2', fontweight='bold', fontsize=11)
ax6.legend()
ax6.grid(True, alpha=0.3, axis='x')

# Subplot 7: Dendrograma jerárquico
ax7 = plt.subplot(3, 3, 7)
linkage_wine = linkage(X_wine_scaled[:50], method='ward')  # Subset para claridad
dendrogram(linkage_wine, ax=ax7)
ax7.set_title('Dendrograma Jerárquico\n(Primeras 50 muestras)', fontweight='bold', fontsize=11)
ax7.set_xlabel('Índice de Muestra')
ax7.set_ylabel('Distancia')
ax7.grid(True, alpha=0.3, axis='y')

# Subplot 8: Heatmap de correlación (top features)
ax8 = plt.subplot(3, 3, 8)
top_10_features = np.argsort(pc1_importance)[-10:]
corr_matrix = np.corrcoef(X_wine[:, top_10_features].T)
sns.heatmap(corr_matrix, annot=False, cmap='coolwarm', center=0, 
           xticklabels=[wine.feature_names[i][:10] for i in top_10_features],
           yticklabels=[wine.feature_names[i][:10] for i in top_10_features],
           ax=ax8, cbar_kws={'label': 'Correlación'})
ax8.set_title('Correlación entre Top 10 Features', fontweight='bold', fontsize=11)
plt.setp(ax8.xaxis.get_majorticklabels(), rotation=45, ha='right', fontsize=8)
plt.setp(ax8.yaxis.get_majorticklabels(), fontsize=8)

# Subplot 9: Comparación métricas clustering
ax9 = plt.subplot(3, 3, 9)
algos = list(clustering_results.keys())
silhouette_vals = [clustering_results[a]['silhouette'] for a in algos]
db_vals = [clustering_results[a]['davies_bouldin'] for a in algos]

x_pos = np.arange(len(algos))
width = 0.35

ax9_twin = ax9.twinx()
bars1 = ax9.bar(x_pos - width/2, silhouette_vals, width, label='Silhouette', alpha=0.8, color='green')
bars2 = ax9_twin.bar(x_pos + width/2, db_vals, width, label='Davies-Bouldin', alpha=0.8, color='orange')

ax9.set_ylabel('Silhouette Score', color='green')
ax9.tick_params(axis='y', labelcolor='green')
ax9_twin.set_ylabel('Davies-Bouldin Index', color='orange')
ax9_twin.tick_params(axis='y', labelcolor='orange')

ax9.set_xticks(x_pos)
ax9.set_xticklabels(algos)
ax9.set_title('Comparación de Métricas\n(Mayor Sil mejor, Menor DB mejor)', fontweight='bold', fontsize=11)
ax9.grid(True, alpha=0.3, axis='y')

# Añadir leyendas
lines1, labels1 = ax9.get_legend_handles_labels()
lines2, labels2 = ax9_twin.get_legend_handles_labels()
ax9.legend(lines1 + lines2, labels1 + labels2, loc='upper left', fontsize=9)

plt.tight_layout()
plt.show()

# Resumen final
print("\n" + "="*80)
print("📊 RESUMEN DEL ANÁLISIS NO SUPERVISADO")
print("="*80)

best_clustering = max(clustering_results.items(), key=lambda x: x[1]['silhouette'])
print(f"\n🏆 Mejor algoritmo de clustering: {best_clustering[0]}")
print(f"   • Silhouette Score: {best_clustering[1]['silhouette']:.4f}")
print(f"   • Davies-Bouldin Index: {best_clustering[1]['davies_bouldin']:.4f}")

print(f"\n📉 Reducción de dimensionalidad (PCA):")
print(f"   • {n_comp_95_wine} componentes capturan 95% de la varianza")
print(f"   • PC1 explica {pca_wine.explained_variance_ratio_[0]*100:.2f}% de la varianza")
print(f"   • Top feature en PC1: {wine.feature_names[np.argmax(np.abs(pca_2d_wine.components_[0]))]}")

print("\n💡 Conclusiones:")
print("   ✅ Los algoritmos de clustering identifican estructuras consistentes con cultivares")
print("   ✅ PCA reduce efectivamente la dimensionalidad preservando información")
print("   ✅ Las características químicas permiten discriminar entre vinos")
print("   ✅ Validación con ground truth confirma calidad de los clusters descubiertos")

## 10. Conclusiones y Mejores Prácticas

### 📚 Resumen de Conceptos Clave

1. **Clustering**:
   - **K-Means**: Rápido, eficiente, pero limitado a clusters esféricos
   - **Jerárquico**: Visualización con dendrogramas, sin necesidad de especificar K
   - **DBSCAN**: Detecta formas arbitrarias y outliers
   - **GMM**: Soft clustering probabilístico con fundamento estadístico

2. **Reducción de Dimensionalidad**:
   - **PCA**: Transformación lineal que maximiza varianza
   - **t-SNE**: Visualización no lineal preservando estructura local

3. **Detección de Anomalías**:
   - **Isolation Forest**: Escalable, eficiente para outliers globales
   - **LOF**: Detecta anomalías locales considerando densidad

### ✅ Mejores Prácticas

#### Pre-procesamiento:
- **Normalización**: Esencial para algoritmos basados en distancia (K-Means, DBSCAN, PCA)
- **Análisis Exploratorio**: Visualizar distribuciones antes de aplicar algoritmos
- **Manejo de Outliers**: Considerar detección y tratamiento antes de clustering

#### Selección de Algoritmos:
- **Clustering**:
  - K-Means: Clusters esféricos, datasets grandes
  - DBSCAN: Formas irregulares, presencia de ruido
  - Jerárquico: Visualización de jerarquías, datasets pequeños/medianos
  - GMM: Necesidad de probabilidades, clusters elípticos
  
- **Reducción Dimensional**:
  - PCA: Pre-procesamiento, compresión, análisis exploratorio rápido
  - t-SNE: Visualización final de alta calidad

#### Evaluación:
- **Métricas Internas**: Silhouette, Davies-Bouldin, Calinski-Harabasz
- **Validación con Dominio**: Interpretar clusters con conocimiento experto
- **Estabilidad**: Probar con diferentes inicializaciones y parámetros

#### Selección de Hiperparámetros:
- **K-Means**: Elbow method, Silhouette analysis para K óptimo
- **DBSCAN**: Análisis de k-distance plot para ε
- **PCA**: Scree plot, varianza acumulada (típicamente 90-95%)
- **t-SNE**: Experimentar con perplexity (5-50)

### 🎯 Ejercicios Propuestos

1. **Ejercicio 1**: Aplica clustering al Iris dataset y compara resultados con etiquetas reales
2. **Ejercicio 2**: Usa PCA para comprimir imágenes (MNIST) y evalúa reconstrucción vs componentes
3. **Ejercicio 3**: Implementa detección de fraude con Isolation Forest en dataset sintético
4. **Ejercicio 4**: Compara t-SNE con diferentes perplexities en dataset de alta dimensión
5. **Ejercicio 5**: Segmentación de clientes usando K-Means + PCA en datos de e-commerce

### 🚀 Aplicaciones Reales

- **Segmentación de Clientes**: Identificar grupos de comportamiento similar
- **Compresión de Datos**: Reducir almacenamiento preservando información
- **Detección de Fraude**: Identificar transacciones anómalas
- **Análisis de Imágenes**: Segmentación, compresión, feature extraction
- **Bioinformática**: Clustering de genes, reducción de dimensionalidad en genómica
- **Sistemas de Recomendación**: Agrupamiento de usuarios/productos similares

### 📖 Recursos Adicionales

- **Scikit-learn User Guide**: https://scikit-learn.org/stable/unsupervised_learning.html
- **Libro**: "Pattern Recognition and Machine Learning" by Christopher Bishop
- **Paper t-SNE**: Van der Maaten & Hinton (2008) - "Visualizing Data using t-SNE"
- **DBSCAN Original**: Ester et al. (1996) - "A Density-Based Algorithm"
- **Kaggle**: Datasets y competiciones de clustering y anomaly detection

### ⚠️ Cuidados y Limitaciones

- **Interpretabilidad**: Clusters no siempre tienen significado intrínseco
- **Escalabilidad**: Algunos algoritmos (jerárquico, t-SNE) no escalan bien
- **Sensibilidad**: Resultados pueden variar con normalización, inicialización
- **Evaluación**: Difícil sin ground truth, requiere métricas múltiples
- **Curse of Dimensionality**: Distancias pierden significado en alta dimensionalidad

---

**🎓 Siguiente Notebook**: [Validación y Evaluación](../03_Validacion_evaluacion/validacion_evaluacion.ipynb)

En el siguiente módulo exploraremos técnicas rigurosas de validación y evaluación de modelos de ML.